In [7]:
import os
from pathlib import Path
import json
import pandas as pd

from eee.solar.model import Location

import sys
sys.path.insert(0,str(Path(os.path.abspath('.')).parent.parent))
from library.solar_resource_api.request_data import HistoricalRequest

## read data

In [2]:
ticket_path = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis/IN3136_EURAC_shared_folder/data_sites"


In [3]:
sites_metadata = pd.read_csv(os.path.join(ticket_path, "sites_metadata.csv"), index_col=0)
sites_metadata

,country,latitude,longitude,altitude,provider
station_name,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn
carpentras,France,44.08,5.06,100.0,bsrn
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn
magurele_mars,Romania,44.34,26.01,110.0,bsrn
cabauw,Netherlands,51.97,4.93,0.0,bsrn
...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc
gonzaga,Italy,44.96,10.77,16.0,lombardy


## Clearness

In [4]:
SOLAR_RESOURCE_KEY = os.environ["EEE_SOLAR_API_LEGACY_KEY"]
solar_resource_url = "https://api.3elabs.eu/solardata?"

In [5]:
times = pd.date_range(start="2022-01-01", end="2024-01-01", freq="H")


In [12]:
for index, row in sites_metadata.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    altitude = row['altitude']

    # API Retrieval
    try:
        historical_request = HistoricalRequest(
            latitude=latitude,
            longitude=longitude,
            variables=["global_horizontal"],
            resolution="h",
            start="2022-01-01",
            end="2024-01-01"
        )

        prod_results_json, execution_time = historical_request.legacy_post_and_get_results(
            token=SOLAR_RESOURCE_KEY,
            url=solar_resource_url
        )
        print(f"Request for site {index} API completed")
        
        # Convert API data to DataFrame
        api_data = pd.DataFrame(prod_results_json["data"], index=pd.DatetimeIndex(prod_results_json["index"]), columns=prod_results_json["columns"])
        api_sum = api_data.sum()
    except Exception as e:
        print(f"Error retrieving API data for site {index}: {str(e)}")
        continue

    # Clearsky Retrieval
    try:
        location = Location(latitude=latitude, longitude=longitude, altitude=altitude)
        clearsky = location.get_irre_clearsky(times=times)
        print(f"Request for site {index} Clearsky completed")
        
        # Ensure clearsky data has datetime index
        clearsky_data = clearsky["ghi"]
        clearsky_data.index = times
        
        # Sum clearsky data
        clearsky_sum = clearsky_data.sum()
    except Exception as e:
        print(f"Error retrieving Clearsky data for site {index}: {str(e)}")
        continue

    # Calculation
    if api_sum is not None and clearsky_sum is not None and clearsky_sum != 0:
        clearness = (api_sum / clearsky_sum) * 100
    else:
        clearness = None
    print(f"Calculation for site {index} completed")

    # Update metadata with clearness value
    sites_metadata.at[index, "clearness (%)"] = clearness



Request for site camborne API completed
Request for site camborne Clearsky completed
Calculation for site camborne completed
Request for site carpentras API completed
Request for site carpentras Clearsky completed
Calculation for site carpentras completed
Request for site budapest_lorinc API completed
Request for site budapest_lorinc Clearsky completed
Calculation for site budapest_lorinc completed
Request for site magurele_mars API completed
Request for site magurele_mars Clearsky completed
Calculation for site magurele_mars completed
Request for site cabauw API completed
Request for site cabauw Clearsky completed
Calculation for site cabauw completed
Request for site cener API completed
Request for site cener Clearsky completed
Calculation for site cener completed
Request for site payerne API completed
Request for site payerne Clearsky completed
Calculation for site payerne completed
Request for site heino API completed
Request for site heino Clearsky completed
Calculation for site h

In [18]:
sites_metadata

,country,latitude,longitude,altitude,provider,clearness (%)
station_name,,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn,68.509617
carpentras,France,44.08,5.06,100.0,bsrn,84.703463
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn,73.573263
magurele_mars,Romania,44.34,26.01,110.0,bsrn,76.351207
cabauw,Netherlands,51.97,4.93,0.0,bsrn,70.044037
...,...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc,NaN
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc,NaN
gonzaga,Italy,44.96,10.77,16.0,lombardy,NaN
